# LAB | Two Sample Hypothesis Tests

### Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd



## Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [3]:
# Run this code:
from google.colab import files
uploaded = files.upload()

Saving Pokemon.csv to Pokemon (1).csv


Let's start off by looking at the `head` function in the cell below.

In [4]:
# Your code here:
pokemon = pd.read_csv('Pokemon.csv')
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [5]:
legendary_counts = pokemon["Legendary"].value_counts().reset_index()

legendary_counts.columns = ["Legendary", "Count"]

print(legendary_counts)

   Legendary  Count
0      False    735
1       True     65


Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [15]:
# Your code here:
legendary_stats = pokemon.groupby("Legendary")["Total"].agg(["count", "mean", "std"]).round(2).reset_index()
legendary_stats.columns = ["Legendary", "Count", "Mean", "Std"]
print(legendary_stats)

total_legendary = pokemon[pokemon["Legendary"] == True]["Total"]
total_non_legendary = pokemon[pokemon["Legendary"] == False]["Total"]

   Legendary  Count    Mean     Std
0      False    735  417.21  106.76
1       True     65  637.38   60.94


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [20]:
from scipy.stats import ttest_ind

# H0: total points for legendary = total points for non-legendary Pokémon
# H1: total points for legendary ≠ total points for non-legendary Pokémon

t_stat, p_value = ttest_ind(total_legendary, total_non_legendary, equal_var=True, alternative='two-sided')
print(f"p-value: {p_value:.4f}")

p-value: 0.0000


What do you conclude from this test? Write your conclusions below.

In [21]:
# Your conclusions here:

alpha = 0.05

if p_value < alpha:
    print("H0 is rejected. The total points for legendary and non-legendary Pokémon are significantly different.")
else:
    print("H0 is not rejected. There is no significant difference in total points.")

H0 is rejected. The total points for legendary and non-legendary Pokémon are significantly different.


How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [28]:
# Your code here:
pokemon_type_count = pokemon.groupby("Type 1")["Total"].agg(["count"]).reset_index()
pokemon_type_sorted = pokemon_type_count.sort_values(by="count", ascending=False)
pokemon_type_sorted

,Type 1,count
17,Water,112
12,Normal,98
9,Grass,70
0,Bug,69
14,Psychic,57
6,Fire,52
3,Electric,44
15,Rock,44
10,Ground,32
2,Dragon,32


Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [32]:
# Your code here:
pokemon_type_stats = pokemon.groupby("Type 1")["Total"].agg(["count", "mean", "std"]).round(0).reset_index()
pokemon_type_stats.sort_values(by="count", ascending=False)

,Type 1,count,mean,std
17,Water,112,430.0,113.0
12,Normal,98,402.0,116.0
9,Grass,70,421.0,107.0
0,Bug,69,379.0,118.0
14,Psychic,57,476.0,139.0
6,Fire,52,458.0,110.0
3,Electric,44,443.0,106.0
15,Rock,44,454.0,108.0
10,Ground,32,438.0,124.0
2,Dragon,32,551.0,146.0


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal.

In [36]:
water_pokemon = pokemon[pokemon["Type 1"] == "Water"]["Total"]
non_water_pokemon = pokemon[pokemon["Type 1"] != "Water"]["Total"]

t_stat, p_value = ttest_ind(water_pokemon, non_water_pokemon, equal_var=True, alternative='two-sided')

# H0: total points of water pokemon = total points of non water pokemon
# H1: total points of water pokemon ≠ total points of non water pokemon

alpha = 0.05

print(f"p-value: {p_value:.4f}")

p-value: 0.6587
H0 is not rejected: There is no significant difference between the total points of water and non water pokemon. They are statistically the same.


Write your conclusion below.

In [37]:
# Your conclusions here:

if p_value < alpha:
    print("H0 is rejected: There is a significant difference between the total points of water and non water pokemon.")
else:
    print("H0 is not rejected: There is no significant difference between the total points of water and non water pokemon. They are statistically the same.")


H0 is not rejected: There is no significant difference between the total points of water and non water pokemon. They are statistically the same.


# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [39]:
# Your code here:
from scipy.stats import ttest_rel

# H0: score_defense = score_attack
# H1: score_defense /= score_attack

pokemon_attack = pokemon["Attack"]
pokemon_defense = pokemon["Defense"]

t_stat, p_value = ttest_rel(pokemon_attack, pokemon_defense)

alpha = 0.05

print(f"p-value: {p_value:.4f}")

p-value: 0.0000


Describe the results of the test in the cell below.

In [41]:
# Your conclusions here:

if p_value < alpha:
    print("H0 is rejected: There is a significant difference between the defense and the attack scores of pokemon.")
else:
    print("H0 is not rejected: There is no significant difference between the defense and attack scores of pokemon. They are statistically the same.")

H0 is rejected: There is a significant difference between the defense and the attack scores of pokemon.


We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below.

In [42]:
# Your code here:

# H0: score_special_defense = score_special_attack
# H1: score_special_defense /= score_special_attack

pokemon_special_attack = pokemon["Sp. Atk"]
pokemon_special_defense = pokemon["Sp. Def"]

t_stat, p_value = ttest_rel(pokemon_special_attack, pokemon_special_defense)

alpha = 0.05

print(f"p-value: {p_value:.4f}")

p-value: 0.3934


Describe the results of the test in the cell below.

In [43]:
# Your conclusions here:

if p_value < alpha:
    print("H0 is rejected: There is a significant difference between the special defense and the attack scores of pokemon.")
else:
    print("H0 is not rejected: There is no significant difference between the special defense and attack scores of pokemon. They are statistically the same.")

H0 is not rejected: There is no significant difference between the special defense and attack scores of pokemon. They are statistically the same.


As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [50]:
# Your code here:
from scipy.stats import ttest_1samp

difference = pokemon["Defense"] - pokemon["Attack"]

# H0: mean difference = 0
# H1: mean difference ≠ 0

t_stat, p_value = ttest_1samp(difference, 0)

alpha = 0.05

print(p_value.round())

if p_value < alpha:
    print("H0 is rejected: The mean difference between defense and attack is significantly different from 0.")
else:
    print("H0 is not rejected: The mean difference between defense and attack is not significantly different from 0.")


0.0
H0 is rejected: The mean difference between defense and attack is significantly different from 0.


# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [57]:
crosstab = pd.crosstab(pokemon["Legendary"], pokemon["Type 1"] == "Water",
                       rownames=['Legendary Status'], colnames=['Is Water'])
print(crosstab)

Is Water          False  True 
Legendary Status              
False               627    108
True                 61      4


Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [59]:
# Your code here:
from scipy.stats import chi2_contingency

chi2, p, dof, expected = chi2_contingency(crosstab)

print(f"Chi²: {chi2:.4f}")
print(f"p-value: {p:.4f}")
print(f"DoF: {dof}")
print("expected frequency:")
print(expected)

Chi²: 2.9429
p-value: 0.0863
DoF: 1
expected frequency:
[[632.1 102.9]
 [ 55.9   9.1]]


Based on a 95% confidence, should we reject the null hypothesis?

In [61]:
# Your answer here:
alpha = 0.05
if p < alpha:
    print('\n→ H0 is rejected: There is a significant relationship between the variables "legendary" and "water".')
else:
    print('\n→ H0 is not rejected: There is NO significant relationship between the variables "legendary" and "water".')



→ H0 is not rejected: There is NO significant relationship between the variables "legendary" and "water".
